In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
import os

np.random.seed(42)

In [ ]:
n_threads = 10

In [ ]:
def ber_directed_divisor_graph(n, p):

    np.random.seed() 
    
    g = ig.Graph(n=n, directed=True)
    edges = []
    
    for i in range(1, n + 1):
        j = 2 * i
        while j <= n:
            if np.random.random() < p:
                edges.append((i - 1, j - 1))
            else:
                edges.append((j - 1, i - 1))
            j += i
    
    g.add_edges(edges)
    return g

In [15]:
def process_single_replicate(args):
    """Process a single replicate for given n and p."""
    n, p, rep = args
    
    # Generate graph
    g = ber_directed_divisor_graph(n, p)
    
    # Find strongly connected components
    scc_list = g.connected_components(mode="strong")
    
    # Get sizes
    comp_sizes = scc_list.sizes()
    num_scc = len(comp_sizes)
    size_lcc = max(comp_sizes) if comp_sizes else 0
    
    size_plus_num_rate = (size_lcc + num_scc - 1) / n
    
    return {
        'n': n,
        'p': p,
        'rep': rep,
        'size_lcc': size_lcc,
        'num_scc': num_scc,
        'size_plus_num_rate': size_plus_num_rate
    }

In [ ]:
def main():
    # PARAMETERS 
    n = 2**10
    ps = np.arange(0.005, 0.015 + 0.0001, 0.0001)
    n_replicate = 50
    
    # CPU count for better speed
    n_workers = os.cpu_count() or 10
    
    results_file = f"7_py_small_2to{int(np.log2(n))}_size_lcc_and_num_scc.csv"
    
    if not Path(results_file).exists():
        df_header = pd.DataFrame(columns=['n', 'p', 'rep', 'size_lcc', 'num_scc', 'size_plus_num_rate'])
        df_header.to_csv(results_file, index=False)
    
    start_time = datetime.now()
    
    for p in ps:
        p_start = datetime.now()
        args_list = [(n, p, rep) for rep in range(1, n_replicate + 1)]
        results = []
        
        # Use ProcessPoolExecutor to bypasses GIL
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            futures = {executor.submit(process_single_replicate, args): args for args in args_list}
            
            for future in as_completed(futures):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    print(f"Error processing replicate: {e}")
        
        if results:
            df_results = pd.DataFrame(results)
            df_results = df_results[['n', 'p', 'rep', 'size_lcc', 'num_scc', 'size_plus_num_rate']]
            df_results.to_csv(results_file, mode='a', header=False, index=False)
        
        elapsed = (datetime.now() - p_start).total_seconds()
        print(f"Completed (n,p) = ({n}, {p:.4f}) at {datetime.now()} (took {elapsed:.2f}s)")
    
    total_time = (datetime.now() - start_time).total_seconds()
    print(f"\nTotal execution time: {total_time:.2f}s")

In [17]:
if __name__ == "__main__":
    main()

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore